In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from os import listdir
from random import shuffle
from matplotlib import pyplot as plt

from torch.utils.tensorboard import SummaryWriter
import time
from tqdm import tqdm
from copy import deepcopy
import torchmetrics


In [ ]:
class Solver:
    def train(
        self,
        model,
        optimizer,
        criterion,
        device,
        train_loader,
        epoch,
        metrics,
    ):
        model.train()
        epoch_loss = 0
        start_time = time.time()
        iteration = 0

        for metric in metrics:
            metric.to(device)
            metric.reset()

        bar = tqdm(train_loader)
        for data in bar:
            data = data.to(device)
            optimizer.zero_grad()

            output = model(data)

            loss = criterion(output, data)
            loss.backward()
            optimizer.step()

            for metric in metrics:
                metric.update(output, data)

            epoch_loss += loss.item()
            iteration += 1
            bar.set_postfix({"Loss": format(epoch_loss / iteration, ".6f")})

        metric_values = [metric.compute().item() for metric in metrics]

        print(f"\rTrain Epoch: {epoch}, elapsed time: {time.time() - start_time:.2f}s")
        return epoch_loss, metric_values

    def test(self, *, model, criterion, test_loader, device, metrics):
        model.eval()
        loss = 0

        for metric in metrics:
            metric.to(device)
            metric.reset()

        with torch.no_grad():
            for data in test_loader:
                data = data.to(device)
                output = model(data)

                loss += criterion(output, data).item()

                for metric in metrics:
                    metric.update(output, data)

        metric_values = [metric.compute().item() for metric in metrics]

        return loss, metric_values

    def training(
        self,
        *,
        model,
        optimizer,
        criterion,
        scheduler,
        device,
        train_loader,
        test_loader,
        epochs,
        metrics,
        name_file: str,
        writing=False,
        warmup_steps=-1,
        warmup_scheduler=None,
        tolerance=-1,
        tolerance_delta=1e-4,
    ):
        writer = SummaryWriter(comment=f"{name_file}")

        best_metric_value = [0.0]
        not_improving = 0
        last_loss = None

        best_model_wts = deepcopy(model.state_dict())
        best_optimizer = deepcopy(optimizer.state_dict())
        metrics_1, metrics_2 = deepcopy(metrics), deepcopy(metrics)

        for epoch in range(1, epochs + 1):
            train_loss, train_metrics = self.train(
                model=model,
                device=device,
                train_loader=train_loader,
                criterion=criterion,
                optimizer=optimizer,
                epoch=epoch,
                metrics=metrics_1,
            )

            test_loss, test_metrics = self.test(
                model=model,
                device=device,
                test_loader=test_loader,
                criterion=criterion,
                metrics=metrics_2,
            )

            if epoch < warmup_steps:
                warmup_scheduler.step(train_loss)
            else:
                scheduler.step(train_loss)

            if (
                test_metrics[0] > best_metric_value[0]
            ):  # Assuming the first metric is the main one
                best_metric_value = test_metrics
                best_model_wts = deepcopy(model.state_dict())
                best_optimizer = deepcopy(optimizer.state_dict())

            if writing:
                writer.add_scalars(
                    "Loss", {"train": train_loss, "test": test_loss}, epoch
                )

                writer.add_scalars(
                    "Metrics",
                    {"train": train_metrics[0], "test": test_metrics[0]},
                    epoch,
                )

            print(
                f"Train Metric: {train_metrics[0]:.10f}, Test Metric: {test_metrics[0]:.10f}"
            )
            print(f"Train Loss: {train_loss:.610}, Test Loss: {test_loss:.10f}")

            if epoch != 1:
                if abs(train_loss - last_loss) < tolerance_delta:
                    not_improving += 1
                    if not_improving == tolerance:
                        print("Stopping early due to tolerance threshold.")
                        break
                else:
                    not_improving = 0

            last_loss = train_loss

        torch.save(model.state_dict(), f"{name_file}_model.pt")
        model.load_state_dict(best_model_wts)
        torch.save(model.state_dict(), f"{name_file}_best_model.pt")

        torch.save(optimizer.state_dict(), f"{name_file}_optimizer.pt")
        optimizer.load_state_dict(best_optimizer)
        torch.save(optimizer.state_dict(), f"{name_file}_best_optimizer.pt")

        if writing:
            writer.close()

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        # encoding 768 image embedding to latent vector of size 256
        self.encoder = nn.Sequential(
            nn.Linear(768, 700),
            nn.ReLU(),
            nn.Linear(700, 640),
            nn.ReLU(),
            nn.Linear(640, 526),
        )
        # decoding latent vector of size 526 to 768 image embedding
        self.decoder = nn.Sequential(
            nn.Linear(526, 640),
            nn.ReLU(),
            nn.Linear(640, 700),
            nn.ReLU(),
            nn.Linear(700, 768),
        )

    def forward(self, x):
        """
        :param x: input embedding
        :return: reconstructed image
        """
        return self.decoder(self.encoder(x))


class AutoEncoderGELU(nn.Module):
    def __init__(self):
        super().__init__()
        # encoding 768 image embedding to latent vector of size 526
        self.encoder = nn.Sequential(
            nn.Linear(768, 700),
            nn.GELU(),
            nn.Linear(700, 640),
            nn.GELU(),
            nn.Linear(640, 526),
        )
        # decoding latent vector of size 526 to 768 image embedding
        self.decoder = nn.Sequential(
            nn.Linear(526, 640),
            nn.GELU(),
            nn.Linear(640, 700),
            nn.GELU(),
            nn.Linear(700, 768),
        )

    def forward(self, x):
        """
        :param x: input embedding
        :return: reconstructed image
        """
        return self.decoder(self.encoder(x))

In [ ]:
class EmbeddingsDataset(torch.utils.data.Dataset):
    def __init__(self, file_path, scalar: float = 1.0):
        self.embeddings = np.load(file_path)
        # for i in range(len(self.embeddings)):
        #     self.embeddings[i] = torch.tensor(self.embeddings[i], dtype=torch.float32)

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return torch.tensor(self.embeddings[idx], dtype=torch.float32)


dataset = EmbeddingsDataset("../data/embeddings_all/all.npy")

train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, test_size]
)


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)

In [ ]:
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

# model = AutoEncoder().to(device)

In [ ]:
model = AutoEncoderGELU().to(device)

In [ ]:
warmup_steps = -1
epochs = 2000
tolerance = -1

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.AdamW(
    model.parameters(), lr=0.00001, amsgrad=True, weight_decay=0.001
)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)
w_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, last_epoch=warmup_steps, T_max=warmup_steps
)

train_metrics = [torchmetrics.MeanSquaredError()]
test_metrics = [torchmetrics.MeanSquaredError()]

In [ ]:
vector = dataset[0].to(device)

model.eval()
m_vector = model.forward(vector)
for i, j in zip(vector, m_vector):
    print(i.float().numpy(force=True), j.float().numpy(force=True))

In [ ]:
solver = Solver()
solver.training(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    device=device,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=epochs,
    name_file="GELU_v2.1",
    metrics=train_metrics,
    writing=True,
    warmup_steps=warmup_steps,
    warmup_scheduler=w_scheduler,
    tolerance=tolerance,
    tolerance_delta=1e-7,
)